In [1]:
import numpy as np
import numba

Python is a nice scripting object-oriented language but it can run into performance issues. We will see a few examples below. To get better performance, one uses compiled languages, such a C, C++ and Fortran. We will also use the numba python library that allows one to perform "just in time" compilations. We will however explore in more details in this lecture how to compile directly C and Fortran codes. We will see in a next lecture how one can interface these compiled functions directly to python.

Let's start first with a simple example to see how bad python performs when not used properly. We define a simple function that uses a python loop, which is generally a very bad idea with python.

In [2]:
def f_simple(X):
    Y = np.empty_like(X)
    for i in range(len(X)):
        x = X[i]
        Y[i] = x + x**2 + x**3 + x**4 + x**5 + x**6 + x**7 + x**8
    return Y

We create a random numpy array of moderate size.

In [3]:
x=np.random.normal(size=1_000_000)

We finally call the function and time it using the ``%%timeit`` python function.

In [4]:
%%timeit 
f_simple(x)

1.82 s ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We see that it called the timing routine 7 times, each timing using only 1 call to the function. We can change this to see how robust the time measurments are. The standard deviation seems indeed a bit large.

In [5]:
%%timeit -r 4 -n 4
f_simple(x)

1.84 s ± 1.23 ms per loop (mean ± std. dev. of 4 runs, 4 loops each)


We see that the measurement seems now more consistent with a smaller standard deviation. 

Let's now try to use more proper python programming, avoiding using explicit loops, but direct numpy array notations instead.

In [6]:
def f_numpy(X):
    return X + X**2 + X**3 + X**4 + X**5 + X**6 + X**7 + X**8

In [7]:
%%timeit -r 4 -n 4
f_numpy(x)

357 ms ± 963 µs per loop (mean ± std. dev. of 4 runs, 4 loops each)


Wow! It is indeed much faster. Too fast even... Let's use a bigger array.

In [8]:
x=np.random.normal(size=10_000_000)

In [9]:
%%timeit -r 4 -n 4
f_numpy(x)

3.64 s ± 19 ms per loop (mean ± std. dev. of 4 runs, 4 loops each)


These multiple powers are probably slow to evaluate. Let's use a nice trick to avoid having to call these expensive operations.

In [10]:
def f_numpy_2(X):
    return X*(1 + X*(1 + X*(1 + X*(1 + X*(1 + X*(1 + X*(1 + X)))))))

In [11]:
%%timeit -r 4 -n 4
f_numpy_2(x)

143 ms ± 644 µs per loop (mean ± std. dev. of 4 runs, 4 loops each)


Wow! Another dramatic improvement!

We kind of reached the maximum we can do using python alone. We will now try to use a nice python package called ``numba`` that allows one to perform _just in time compilation_. What ``numba`` does is to first convert the python function into a C code and then to compile this C code on the fly. The performance of the resulting function is usually much higher. Since the function is now compiled, you don't need to worry about using loops directly anymore. In fact, to allow ``numba`` to translate the python instructions into C instructions, it is recommended to use explicit loops. 

Let see how we can optimize our function using ``numba``.

In [12]:
@numba.jit(nopython=True)
def f_numba(X):
    Y = np.empty_like(X)
    for i in range(len(X)):
        x = X[i]
        Y[i] = x + x**2 + x**3 + x**4 + x**5 + x**6 + x**7 + x**8
#        Y[i] = x*(1 + x*(1 + x*(1 + x*(1 + x*(1 + x*(1 + x*(1 + x)))))))
    return Y

Note that we have used the _decorator_ ``@numba.jit`` that tells ``numba`` to translate the function in C and compile it. ``numba`` tries to translate everything in C. If it cannot, it will keep the python code as is. Using the option ``nopython=True`` forces ``numba`` to translate in C. If ``numba`` fails to do it, an error will follow.

Let's now time the resulting _compiled_ function.

In [13]:
%%timeit -r 4 -n 4
f_numba(x)

The slowest run took 8.62 times longer than the fastest. This could mean that an intermediate result is being cached.
63.5 ms ± 72.1 ms per loop (mean ± std. dev. of 4 runs, 4 loops each)


This is now really fast! This is the main advantage of using a compiled language. The standard deviation is quite large when compared to the mean. This is because the timer is also counting the extra time ``numba`` needs to compile the function. To avoid this, we can use an even bigger array. Note that we could also have used the ``cache=True`` option of ``numba`` but this is beyond the scope of this lecture. 

In [14]:
x=np.random.normal(size=100_000_000)

In [15]:
%%timeit -r 4 -n 4
f_numba(x)

238 ms ± 476 µs per loop (mean ± std. dev. of 4 runs, 4 loops each)


Using a compiler also allows us to use parallel computing. We will see in future lectures how to program in parallel. For the time being, we just trust ``numba`` to do it for us. To parallelize a ``numba`` function, just add the ``parallel=True`` option and replace the ``range`` function defining the loop by the parallel function ``numba.prange`` which defines the method to divide up the loop into parallel tasks.

In [16]:
@numba.jit(nopython=True, parallel=True)
def f_numba_para(X):
    Y = np.empty_like(X)
    for i in numba.prange(len(X)):
        x = X[i]
        Y[i] = x + x**2 + x**3 + x**4 + x**5 + x**6 + x**7 + x**8
    return Y

In [19]:
%%timeit -r 4 -n 40
f_numba_para(x)

26 ms ± 276 µs per loop (mean ± std. dev. of 4 runs, 40 loops each)


This ends our journey towards better and better performance. We started with an explicit loop within a pure python function and the pretty awful timing of roughly 200s. We ended with a compiled parallel C code generated by ``numba`` with an amazing 10000x (yes 10 thousands!) speedup with roughly 20ms of execution time.  